In [ ]:
from mxnet import gluon
from mxnet import nd
from mxnet import image

train_augs = [
    image.HorizontalFlipAug(0.5),      # 横向变换
    image.RandomCropAug((224, 224))    # 变换图像尺寸
]

test_augs = [
    image.RandomCropAug((224, 224))
]

# data 图像数据， label 图像标识， augs 图像增广内容
def transform(data, label, augs):
    data = data.astype('float32')
    for aug in augs:
        data = aug(data)
    data = nd.transpose(data, (2, 0, 1))  # 通道的变换，最后一通道前置
    return data, nd.array([label]).asscalar().astype('float32')

In [ ]:
import sys
sys.path.append('../../gluon-tutorials-zh')
import utils

data_dir = '../../Data'
train_imgs = gluon.data.vision.ImageFolderDataset(data_dir+'/hotdog/train', transform=lambda X, y:transform(X, y, train_augs))
test_imgs = gluon.data.vision.ImageFolderDataset(data_dir+'/hotdog/test', transform=lambda X, y:transform(X, y, test_augs))
#print(train_imgs.shape)
for Z in train_imgs:
    print(Z)
    break

data = gluon.data.DataLoader(train_imgs, 32, shuffle=True)
#print(data)
# 数据的显示
for X, Y in data:
    # clip函数作用为取0为最小，255为最大，并且被255所除，归一为0-1
    X = X.transpose((0,2,3,1)).clip(0, 255)/255
    # utils.show_images(X, 4, 8)
    # print(X[0][0])
    # print(X.shape)
    # print(Y)
    break



In [ ]:
from mxnet.gluon.model_zoo import vision as models
pretrained_net = models.resnet18_v2(pretrained=True)
# pretrained_net = models.resnet18_v2(pretrained=True)

In [ ]:
print(pretrained_net.classifier)
print(pretrained_net.classifier[4].weight.data())  # 全连接层

In [ ]:
from mxnet import init

# classes为分类的类别种类
finetune_net = models.resnet18_v2(classes=2)
# print(finetune_net)
finetune_net.features = pretrained_net.features
finetune_net.classifier.initialize(init.Xavier())
# print(finetune_net)

### 训练函数

In [ ]:
def train(net, ctx, batch_size=64, epochs=10, learning_rate=0.01, wd=0.001):
    train_data = gluon.data.DataLoader(train_imgs, batch_size, shuffle=True)
    test_data = gluon.data.DataLoader(test_imgs, batch_size)
    # print(train_data)
    # net在ctx上面
    net.collect_params().reset_ctx(ctx)
    net.hybridize()
    loss = gluon.loss.SoftmaxCrossEntropyLoss()
    
    # train
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': learning_rate})
    utils.train(train_data, test_data, net, loss, trainer, ctx, epochs)

In [ ]:
ctx = utils.try_gpu()
train(finetune_net, ctx)

In [ ]:
scratch_net = models.resnet18_v2(classes=2)
scratch_net.initialize(init=init.Xavier())
train(scratch_net, ctx)

In [ ]:
import matplotlib.pyplot as plt

def classify_hotdog(net, fname, ctx):
    with open(fname, 'rb') as f:
        img = image.imdecode(f.read())
        #print(img)
    data, _ = transform(img, -1, test_augs)
    data_cal = data.expand_dims(axis=0)
    out = net(data_cal.as_in_context(ctx))
    out = nd.SoftmaxActivation(out)
    pred = int(nd.argmax(out, axis=1).asscalar())
    prob = out[0][pred].asscalar()
    print(prob)
    label = train_imgs.synsets
    print(label[pred])
    data = nd.transpose(data, (1,2,0)).clip(0, 255)/255
    data = data.asnumpy()
    plt.imshow(data)
    plt.show()
    

In [ ]:
import os
print(os.getcwd())

classify_hotdog(finetune_net, '../../Data/hotdog/test/not-hotdog/1221.png', ctx)